In [71]:
import pandas as pd
import numpy as np

from typing import Callable

In [72]:
data = pd.read_csv("train.csv")

In [116]:
def window_sliding(data: pd.DataFrame, features: dict[str, list[tuple]], in_place=False):
    if not in_place:
        data = data.copy()
    
    for feature, functions in features.items():
        for func, w_size in functions:
            rolling = data.rolling(window=w_size, min_periods=1)
            data[f"{feature}_{func.__name__}_{w_size}h"] = rolling[feature].agg(func)
    
    if not in_place:
        return data

In [117]:
data_cp = data.copy()

In [118]:
window_sliding(data_cp, dict(zip(list(data.columns[-3:]), [[(np.average, 24), (np.max, 12)], 
                                                           [(np.average, 24), (np.max, 6)], 
                                                           [(np.average, 24), (np.max, 24)]])), in_place=True)

In [120]:
data_cp.head(5)

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,target_carbon_monoxide_average_24h,target_carbon_monoxide_amax_12h,target_benzene_average_24h,target_benzene_amax_6h,target_nitrogen_oxides_average_24h,target_nitrogen_oxides_amax_24h
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7,2.500000,2.5,12.000000,12.0,167.700000,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9,2.300000,2.5,10.950000,12.0,133.300000,167.7
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1,2.266667,2.5,10.366667,12.0,131.233333,167.7
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2,2.250000,2.5,10.200000,12.0,142.725000,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8,2.100000,2.5,9.440000,12.0,138.540000,177.2
